In [1]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### XToYHTo6B UL production

In [2]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
                             pythia8CommonSettingsBlock,
                             pythia8CP5SettingsBlock,
                             pythia8PSweightsSettingsBlock,
                             processParameters = cms.vstring(
                             __CHANNEL_DECAY_FRAGMENT__
                             ),
                             parameterSets = cms.vstring('pythia8CommonSettings',
                                                         'pythia8CP5Settings',
                                                         'pythia8PSweightsSettings',
                                                         'processParameters'
                                                     )
                         )
)
ProductionFilterSequence = cms.Sequence(generator)
"""

In [3]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2016APV': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [4]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(True),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [5]:
years = ["2016", "2016APV", "2017", "2018"]
mX = ["400", "450", "500", "550", "600", "650", "700", "750", "800", "850", "900", "950", "1000", "1100", "1200", "1300", "1400", "1500", "1600", "1700", "1800", "1900", "2000", "2200", "2400", "2500", "2600", "2800", "3000", "3500", "4000"]
mY = ["250", "300", "350", "400", "450", "500", "600", "700", "800", "900", "1000", "1100", "1200", "1300", "1400", "1600", "1800", "2000", "2200", "2400", "2500", "2600", "2800"]

tot_events1 = [200000]*len(mY)
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [6]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/NMSSM_XYH_YToHH_MX_1200_MY_500'
    t_datasetname_year = 'NMSSM_XToYHTo6B_MX-{massX}_MY-{massY}_TuneCP5_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/slc7_amd64_gcc700/MadGraph5_aMCatNLO/NMSSM_XToYH/NMSSM_XYH_YToHH_MX_{massX}_MY_{massY}_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    
    dataset_names_year = []
    gp_ggf_year = []

    for mx in mX:
        for my in mY:
            d = {"massX": mx, "massY": my}
            tmp_example_dict = t_example1
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict = dataset_names_year
            tmp_gridpack_dict = gp_ggf_year
        
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

#print(dataset_names['2017'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

In [7]:
x_hy_6b = ''' 
    '25:onMode = off',
    '25:onIfMatch = 5 -5',
    'ResonanceDecayFilter:filter = on'
'''

## .csv Production

In [8]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
tmp_process_pythia_map = fragmentsDictCreator(x_hy_6b)
process_pythia_map.update(tmp_process_pythia_map)

In [9]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    with open('XtoYHto6B' + '_' + year +'.csv', 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',',
                        quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator','mcdbid','time','size'])
        '''if year == "2016":
                if process == "Radion":
                    pr = 1606
                elif process == "BulkGraviton":
                    pr = 1631
            if year == "2016APV":
                if process == "Radion":
                    pr = 812
                elif process == "BulkGraviton":
                    pr = 837
            if year == "2017":
                if process == "Radion":
                    pr = 1574
                elif process == "BulkGraviton":
                    pr = 1599
            if year == "2018":
                if process == "Radion":
                    pr = 1612
                elif process == "BulkGraviton":
                    pr = 1637'''
        k=0
        for i in range(len(mX)):
            for j in range(len(mY)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.6.5"
                dataset_name = dataset_names[year][k]
                if year == "2016APV":
                    if int(mX[i]) == 400 or int(mX[i]) == 500:
                        events = round(400000*0.54)
                    elif int(mX[i]) == 450 and int(mY[j]) == 250:
                        events = round(400000*0.54)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(400000*0.54)
                    elif int(mX[i]) == 600 and int(mY[j]) == 250:
                        events = round(400000*0.54)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 350 or int(mY[j]) == 400):
                        events = round(250000*0.54)
                    elif int(mX[i]) == 600 and (int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 400 or int(mY[j]) == 450):
                        events = round(250000*0.54)
                    elif int(mX[i]) == 650 or int(mX[i]) == 750:
                        events = round(250000*0.54)
                    elif int(mX[i]) == 700 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 450 or int(mY[j]) == 500):
                        events = round(250000*0.54)
                    elif int(mX[i]) == 800 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350):
                        events = round(250000*0.54)
                    elif int(mX[i]) == 850 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(250000*0.54)
                    elif int(mX[i]) == 900 and int(mY[j]) == 250:
                        events = round(250000*0.54)
                    elif int(mX[i]) == 950 and int(mY[j]) == 250:
                        events = round(250000*0.54)
                    elif int(mX[i]) == 450 and int(mY[j]) == 300:
                        events = round(2400000*0.54)
                    elif int(mX[i]) == 700 and int(mY[j]) == 400:
                        events = round(900000*0.54)
                    elif int(mX[i]) == 900 and int(mY[j]) == 300:
                        events = round(840000*0.54)
                    elif int(mX[i]) == 1000 and int(mY[j]) == 600:
                        events = round(720000*0.54)
                    else:
                        events = round(tot_events1[j]*0.54)
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGENAPV-00' + str(pr)
                    
                elif year == "2016":
                    if int(mX[i]) == 400 or int(mX[i]) == 500:
                        events = round(400000*0.46)
                    elif int(mX[i]) == 450 and int(mY[j]) == 250:
                        events = round(400000*0.46)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(400000*0.46)
                    elif int(mX[i]) == 600 and int(mY[j]) == 250:
                        events = round(400000*0.46)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 350 or int(mY[j]) == 400):
                        events = round(250000*0.46)
                    elif int(mX[i]) == 600 and (int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 400 or int(mY[j]) == 450):
                        events = round(250000*0.46)
                    elif int(mX[i]) == 650 or int(mX[i]) == 750:
                        events = round(250000*0.46)
                    elif int(mX[i]) == 700 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 450 or int(mY[j]) == 500):
                        events = round(250000*0.46)
                    elif int(mX[i]) == 800 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350):
                        events = round(250000*0.46)
                    elif int(mX[i]) == 850 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(250000*0.46)
                    elif int(mX[i]) == 900 and int(mY[j]) == 250:
                        events = round(250000*0.46)
                    elif int(mX[i]) == 950 and int(mY[j]) == 250:
                        events = round(250000*0.46)
                    elif int(mX[i]) == 450 and int(mY[j]) == 300:
                        events = round(2400000*0.46)
                    elif int(mX[i]) == 700 and int(mY[j]) == 400:
                        events = round(900000*0.46)
                    elif int(mX[i]) == 900 and int(mY[j]) == 300:
                        events = round(840000*0.46)
                    elif int(mX[i]) == 1000 and int(mY[j]) == 600:
                        events = round(720000*0.46)
                    else:
                        events = round(tot_events1[j]*0.46)
                    #prepid = 'HIG-RunIISummer20UL16wmLHEGEN-0' + str(pr)
                    
                elif year == "2017":
                    if int(mX[i]) == 400 or int(mX[i]) == 500:
                        events = round(400000)
                    elif int(mX[i]) == 450 and int(mY[j]) == 250:
                        events = round(400000)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(400000)
                    elif int(mX[i]) == 600 and int(mY[j]) == 250:
                        events = round(400000)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 350 or int(mY[j]) == 400):
                        events = round(250000)
                    elif int(mX[i]) == 600 and (int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 400 or int(mY[j]) == 450):
                        events = round(250000)
                    elif int(mX[i]) == 650 or int(mX[i]) == 750:
                        events = round(250000)
                    elif int(mX[i]) == 700 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 450 or int(mY[j]) == 500):
                        events = round(250000)
                    elif int(mX[i]) == 800 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350):
                        events = round(250000)
                    elif int(mX[i]) == 850 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(250000)
                    elif int(mX[i]) == 900 and int(mY[j]) == 250:
                        events = round(250000)
                    elif int(mX[i]) == 950 and int(mY[j]) == 250:
                        events = round(250000)
                    elif int(mX[i]) == 450 and int(mY[j]) == 300:
                        events = round(2400000)
                    elif int(mX[i]) == 700 and int(mY[j]) == 400:
                        events = round(900000)
                    elif int(mX[i]) == 900 and int(mY[j]) == 300:
                        events = round(840000)
                    elif int(mX[i]) == 1000 and int(mY[j]) == 600:
                        events = round(720000)
                    else:
                        events = round(tot_events1[j])
                    #prepid = 'HIG-RunIISummer20UL17wmLHEGEN-0' + str(pr)
                elif year == "2018":
                    if int(mX[i]) == 400 or int(mX[i]) == 500:
                        events = round(400000)
                    elif int(mX[i]) == 450 and int(mY[j]) == 250:
                        events = round(400000)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(400000)
                    elif int(mX[i]) == 600 and int(mY[j]) == 250:
                        events = round(400000)
                    elif int(mX[i]) == 550 and (int(mY[j]) == 350 or int(mY[j]) == 400):
                        events = round(250000)
                    elif int(mX[i]) == 600 and (int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 400 or int(mY[j]) == 450):
                        events = round(250000)
                    elif int(mX[i]) == 650 or int(mX[i]) == 750:
                        events = round(250000)
                    elif int(mX[i]) == 700 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350 or int(mY[j]) == 450 or int(mY[j]) == 500):
                        events = round(250000)
                    elif int(mX[i]) == 800 and (int(mY[j]) == 250 or int(mY[j]) == 300 or int(mY[j]) == 350):
                        events = round(250000)
                    elif int(mX[i]) == 850 and (int(mY[j]) == 250 or int(mY[j]) == 300):
                        events = round(250000)
                    elif int(mX[i]) == 900 and int(mY[j]) == 250:
                        events = round(250000)
                    elif int(mX[i]) == 950 and int(mY[j]) == 250:
                        events = round(250000)
                    elif int(mX[i]) == 450 and int(mY[j]) == 300:
                        events = round(2400000)
                    elif int(mX[i]) == 700 and int(mY[j]) == 400:
                        events = round(900000)
                    elif int(mX[i]) == 900 and int(mY[j]) == 300:
                        events = round(840000)
                    elif int(mX[i]) == 1000 and int(mY[j]) == 600:
                        events = round(720000)
                    else:
                        events = round(tot_events1[j])
                    #prepid = 'HIG-RunIISummer20UL18wmLHEGEN-0' + str(pr)
                
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][k]) + '\n' + process_pythia_map[year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + "  Pythia8"
                mcdb_id = '0'
                time = '2.2'
                size = '132'
                if int(mX[i]) > (int(mY[j]) + 125) and int(mY[j]) >= 2*125:
                    csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])
                #pr = pr+1   
                k=k+1